In [1]:
import cv2
import copy
from ultralytics import solutions
import os

def save_cropped_box(im0, box, track_id, frame_count, output_dir="videos/crop"):
    """Saves the cropped image of the bounding box as a separate JPG file."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Convert the box coordinates to integers
    x1, y1, x2, y2 = map(int, box)  # Ensure the coordinates are integers
    cropped_box = im0[y1:y2, x1:x2]  # Crop the image using the bounding box coordinates
    output_path = f"{output_dir}/track_{track_id}_frame_{frame_count}.jpg"
    cv2.imwrite(output_path, cropped_box)  # Save t
   
def make_hashable(dict_items):
    return {(k, tuple(v.items()) if isinstance(v, dict) else v) for k, v in dict_items}
    
def save_detection_frame(frame, output_dir, frame_count):
    cv2.imwrite(f"{output_dir}/frame_{frame_count}.jpg", frame)

def count_specific_classes(video_path, output_video_path, model_path, classes_to_count):
    """Count specific classes of objects in a video."""
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    #line_points = [(20, 500), (1500, 150)]
    line_points= [(10,600),(1300,0)]
    counter = solutions.ObjectCounter(show=True, region=line_points, model=model_path, classes=classes_to_count)
    frame_count = 0
    #couter_copy=copy.deepcopy(counter.classwise_counts.items())
    #couter_copy_items= counter.classwise_counts.items()
    previous_items = make_hashable(counter.classwise_counts.items())
    previous_positions = {}  # Store previous positions of objects
    objects_crossed = set()  # Track objects that have crossed the line

    # Get the equation of the line (y = mx + b) using the two points
    x1, y1 = line_points[0]
    x2, y2 = line_points[1]
    m = (y2 - y1) / (x2 - x1)  # Slope
    b = y1 - m * x1  # Intercept
    
    while cap.isOpened():
        success, im0 = cap.read()
            
        if not success:
            print("Video frame is empty or video processing has been successfully completed.")
            break

        im0 = counter.count(im0)        
        video_writer.write(im0)
        
        #if counter.classwise_counts.update():
        frame_count+=1
        current_items = make_hashable(counter.classwise_counts.items())

        added = current_items - previous_items
        removed = previous_items - current_items
        if added or removed:
            print(counter.classwise_counts.items())
            save_detection_frame(im0, "videos/", frame_count)
            previous_items = current_items   

            for track_id, box in zip(counter.track_ids, counter.boxes):
                x1, y1, x2, y2 = box
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)

                if track_id in previous_positions:
                    prev_centroid = previous_positions[track_id]

                    # Check if the object crossed the oblique line
                    prev_y = prev_centroid[1]
                    current_y = centroid[1]

                    # Calculate the y value of the line at the object's x position
                    line_y_at_x = m * centroid[0] + b

                    # If the current centroid's Y is above or below the line, it means it crossed
                    if (prev_y > line_y_at_x and current_y <= line_y_at_x) or \
                       (prev_y < line_y_at_x and current_y >= line_y_at_x):
                        if track_id not in objects_crossed:
                            print(f"Object {track_id} crossed the line! Bounding Box: {box}")
                            save_cropped_box(im0, box, track_id, frame_count)
                            objects_crossed.add(track_id)

                previous_positions[track_id] = centroid    
             
  
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

    print("Resumen --------------------------")
 # Imprime los totales
    # Inicializa un diccionario para los totales
    total_counts = {}

    # Itera sobre cada tipo de vehículo en el diccionario
    for vehicle, data in counter.classwise_counts.items():
        # Suma las entradas y salidas
        total = data['IN'] + data['OUT']
    
    # Guarda el total en el nuevo diccionario
        total_counts[vehicle] = total
    print("Listado Final :")
    # Imprime los totales por tipo
    for vehicle, total in total_counts.items():
        print(f"{vehicle.capitalize()}s: {total}")

#//classes Indentificadas por yolo ,
 #   "1": "bicycle",
 #  "2": "car",
 # "3": "motorcycle",
 #"4": "airplane",
 #"5": "bus",
 #"7": "truck",


#count_specific_classes("videos/ejemplo0.mkv", "output_specific_classes.avi", "yolo11n.pt", [1,2,3,5,7])
count_specific_classes("videos/ejemplo0.mp4", "output_specific_classes.avi", "yolo11x.pt", [1,2,3,5,7])

Ultralytics Solutions:  {'region': [(10, 600), (1300, 0)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'show': True, 'model': 'yolo11x.pt', 'classes': [1, 2, 3, 5, 7]}

0: 384x640 9 cars, 2 trucks, 125.0ms
Speed: 3.0ms preprocess, 125.0ms inference, 56.0ms postprocess per image at shape (1, 3, 384, 640)
dict_items([('truck', {'IN': 0, 'OUT': 0}), ('car', {'IN': 0, 'OUT': 0})])

0: 384x640 8 cars, 2 trucks, 119.0ms
Speed: 1.0ms preprocess, 119.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 2 trucks, 114.0ms
Speed: 2.0ms preprocess, 114.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 2 trucks, 113.0ms
Speed: 2.0ms preprocess, 113.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 2 trucks, 115.0ms
Speed: 2.0ms preprocess, 115.0ms inference, 1.0ms postproces

In [ ]:
counter.Polygon.xy()